In [1]:
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm

In [2]:
# TRANSFER_PATH = r"D:\Downloads\Background Matting\dataset\VideoMatte20KFrames\valid"
# CONVLSTM_PATH = r"D:\Downloads\Background Matting\dataset\VideoMatte20KFrames\validConvLSTM"
TRANSFER_PATH = r"D:\Downloads\Background Matting\dataset\VideoMatte20KFrames\valid2"
CONVLSTM_PATH = r"D:\Downloads\Background Matting\dataset\VideoMatte20KFrames\validConvLSTM2"
ORIGINAL_PATH = r"D:\Downloads\Background Matting\dataset\VideoMatte20KFrames\validOriginal"

In [3]:
def read_img(pathname, mode="L"):
    with Image.open(pathname) as img:
        return img.convert(mode)

In [4]:
def get_img_list(path, mode="L"):
    return [read_img(os.path.join(path, name), mode) for name in os.listdir(path)]

### MAD (mean absolute difference)

In [5]:
class MetricMAD:
    def __call__(self, pred, true):
        return np.abs(pred - true).mean()

### MSE (mean squared error)

In [6]:
class MetricMSE:
    def __call__(self, pred, true):
        return ((pred - true) ** 2).mean()

### GRAD

In [7]:
class MetricGRAD:
    def __init__(self, sigma=1.4):
        self.filter_x, self.filter_y = self.gauss_filter(sigma)

    def __call__(self, pred, true):
        pred_normed = np.zeros_like(pred)
        true_normed = np.zeros_like(true)
        cv2.normalize(pred, pred_normed, 1.0, 0.0, cv2.NORM_MINMAX)
        cv2.normalize(true, true_normed, 1.0, 0.0, cv2.NORM_MINMAX)

        true_grad = self.gauss_gradient(true_normed).astype(np.float32)
        pred_grad = self.gauss_gradient(pred_normed).astype(np.float32)

        grad_loss = ((true_grad - pred_grad) ** 2).sum()
        return grad_loss / 1000

    def gauss_gradient(self, img):
        img_filtered_x = cv2.filter2D(img, -1, self.filter_x, borderType=cv2.BORDER_REPLICATE)
        img_filtered_y = cv2.filter2D(img, -1, self.filter_y, borderType=cv2.BORDER_REPLICATE)
        return np.sqrt(img_filtered_x ** 2 + img_filtered_y ** 2)

    @staticmethod
    def gauss_filter(sigma, epsilon=1e-2):
        half_size = np.ceil(sigma * np.sqrt(-2 * np.log(np.sqrt(2 * np.pi) * sigma * epsilon)))
        size = np.int(2 * half_size + 1)

        # create filter in x axis
        filter_x = np.zeros((size, size))
        for i in range(size):
            for j in range(size):
                filter_x[i, j] = MetricGRAD.gaussian(i - half_size, sigma) * MetricGRAD.dgaussian(j - half_size, sigma)

        # normalize filter
        norm = np.sqrt((filter_x ** 2).sum())
        filter_x = filter_x / norm
        filter_y = np.transpose(filter_x)

        return filter_x, filter_y

    @staticmethod
    def gaussian(x, sigma):
        return np.exp(-(x ** 2) / (2 * sigma ** 2)) / (sigma * np.sqrt(2 * np.pi))

    @staticmethod
    def dgaussian(x, sigma):
        return -x * MetricGRAD.gaussian(x, sigma) / sigma ** 2

### CONN

In [8]:
class MetricCONN:
    def __call__(self, pred, true):
        step = 0.1
        thresh_steps = np.arange(0, 1 + step, step)
        round_down_map = -np.ones_like(true)
        for i in range(1, len(thresh_steps)):
            true_thresh = true >= thresh_steps[i]
            pred_thresh = pred >= thresh_steps[i]
            intersection = (true_thresh & pred_thresh).astype(np.uint8)

            # connected components
            _, output, stats, _ = cv2.connectedComponentsWithStats(intersection, connectivity=4)
            # start from 1 in dim 0 to exclude background
            size = stats[1:, -1]

            # largest connected component of the intersection
            omega = np.zeros_like(true)
            if len(size) != 0:
                max_id = np.argmax(size)
                # plus one to include background
                omega[output == max_id + 1] = 1

            mask = (round_down_map == -1) & (omega == 0)
            round_down_map[mask] = thresh_steps[i - 1]
        round_down_map[round_down_map == -1] = 1

        true_diff = true - round_down_map
        pred_diff = pred - round_down_map
        # only calculate difference larger than or equal to 0.15
        true_phi = 1 - true_diff * (true_diff >= 0.15)
        pred_phi = 1 - pred_diff * (pred_diff >= 0.15)

        connectivity_error = np.sum(np.abs(true_phi - pred_phi))
        return connectivity_error / 1000

In [9]:
metric_list = [MetricMAD(), MetricMSE(), MetricGRAD(), MetricCONN()]

In [10]:
def get_result_list(path):
    pred_pha_list = get_img_list(os.path.join(path, "pred_pha"))
    gt_pha_list = get_img_list(os.path.join(path, "true_pha"))
    ret = []
    for pred_pha, true_pha in zip(pred_pha_list, gt_pha_list):
        ret.append([metric(np.array(pred_pha), np.array(true_pha)) for metric in metric_list])
    return ret

In [11]:
transfer_result_list = get_result_list(TRANSFER_PATH)
convSLTM_result_list = get_result_list(CONVLSTM_PATH)
original_result_list = get_result_list(ORIGINAL_PATH)

In [22]:
transfer = pd.DataFrame(transfer_result_list, columns=["MAD", "MSE", "GRAD", "CONN"]).mean()
transfer

MAD       8.093385
MSE       5.442810
GRAD      6.728481
CONN    406.093677
dtype: float64

In [25]:
convLSTM = pd.DataFrame(convSLTM_result_list, columns=["MAD", "MSE", "GRAD", "CONN"]).mean()
convLSTM

MAD       5.175114
MSE       3.730596
GRAD      7.650366
CONN    259.666536
dtype: float64

In [26]:
original = pd.DataFrame(original_result_list, columns=["MAD", "MSE", "GRAD", "CONN"]).mean()
original

MAD       2.040768
MSE       0.942249
GRAD      0.114096
CONN    102.397561
dtype: float64

In [ ]:
result = pd.DataFrame(dict(original=original, transfer=transfer, convLSTM=convLSTM)).T
result

In [35]:
result = pd.DataFrame(dict(original=original, transfer=transfer, convLSTM=convLSTM)).T
result

,MAD,MSE,GRAD,CONN
original,2.040768,0.942249,0.114096,102.397561
transfer,8.093385,5.442810,6.728481,406.093677
convLSTM,5.175114,3.730596,7.650366,259.666536


In [41]:
print(result.to_latex(float_format="%.2f", caption="caption", label="result", column_format="lcccc"))

\begin{table}
\centering
\caption{caption}
\label{result}
\begin{tabular}{lcccc}
\toprule
{} &  MAD &  MSE &  GRAD &   CONN \\
\midrule
original & 2.04 & 0.94 &  0.11 & 102.40 \\
transfer & 8.09 & 5.44 &  6.73 & 406.09 \\
convLSTM & 5.18 & 3.73 &  7.65 & 259.67 \\
\bottomrule
\end{tabular}
\end{table}



In [36]:
result.to_latex?

Signature:
result.to_latex(
    buf=None,
    columns=None,
    col_space=None,
    header=True,
    index=True,
    na_rep='NaN',
    formatters=None,
    float_format=None,
    sparsify=None,
    index_names=True,
    bold_rows=False,
    column_format=None,
    longtable=None,
    escape=None,
    encoding=None,
    decimal='.',
    multicolumn=None,
    multicolumn_format=None,
    multirow=None,
    caption=None,
    label=None,
    position=None,
)
Docstring:
Render object to a LaTeX tabular, longtable, or nested table/tabular.

Requires ``\usepackage{booktabs}``.  The output can be copy/pasted
into a main LaTeX document or read from an external file
with ``\input{table.tex}``.

.. versionchanged:: 1.0.0
   Added caption and label arguments.

.. versionchanged:: 1.2.0
   Added position argument, changed meaning of caption argument.

Parameters
----------
buf : str, Path or StringIO-like, optional, default None
    Buffer to write to. If None, the output is returned as a string.
c